# France Wheat Futures Prediction - Complete Pipeline
# 
# This notebook:
# 1. Loads CY-Bench agricultural data (weather, vegetation, soil)
# 2. Loads Euronext wheat futures prices
# 3. Interpolates sparse data to daily resolution
# 4. Creates t+20 day price prediction targets
# 5. Comprehensive data exploration
# 6. Saves processed dataset for modeling

In [ ]:
# Core imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✓ Imports successful")

In [ ]:
# ============================================================
# CONFIGURATION
# ============================================================

# Paths
CYBENCH_DIR = Path("../data/raw/cybench/cybench-data/wheat/FR/")
FUTURES_PATH = Path("../data/raw/futures/milling_wheat_raw.csv")
OUTPUT_DIR = Path("../data/processed")
FIGURES_DIR = Path("../reports/figures")

# Create directories
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

# Parameters
HORIZON = 20  # Predict price 20 trading days ahead
SAMPLE_SIZE = 10000  # Sample size for quick testing
RANDOM_SEED = 42

print("✓ Configuration set")
print(f"  - Prediction horizon: t+{HORIZON} days")
print(f"  - Output directory: {OUTPUT_DIR}")
print(f"  - Figures directory: {FIGURES_DIR}")

## 1. Data Loading

In [ ]:
# ============================================================
# LOAD CY-BENCH DATA
# ============================================================

print("Loading CY-Bench France wheat data...\n")

# 1. Yield data (ground truth for original task)
yield_df = pd.read_csv(CYBENCH_DIR / "yield_wheat_FR.csv")
print(f"✓ Yield data: {yield_df.shape}")
print(f"  Years: {yield_df['harvest_year'].min()}-{yield_df['harvest_year'].max()}")
print(f"  Regions: {yield_df['adm_id'].nunique()}")

# 2. Weather data (daily)
meteo_df = pd.read_csv(CYBENCH_DIR / "meteo_wheat_FR.csv")
print(f"\n✓ Weather data: {meteo_df.shape}")
print(f"  Columns: {list(meteo_df.columns)}")

# 3. Vegetation data (dekadal - 10-day periods)
fpar_df = pd.read_csv(CYBENCH_DIR / "fpar_wheat_FR.csv")
print(f"\n✓ fPAR data: {fpar_df.shape}")

# 4. Soil moisture (if available)
try:
    soil_df = pd.read_csv(CYBENCH_DIR / "soil_moisture_wheat_FR.csv")
    print(f"\n✓ Soil moisture data: {soil_df.shape}")
except FileNotFoundError:
    print("\n⚠️  Soil moisture data not found")
    soil_df = None

# 5. Crop calendar (static - growing season dates)
try:
    calendar_df = pd.read_csv(CYBENCH_DIR / "crop_calendar_wheat_FR.csv")
    print(f"\n✓ Crop calendar: {calendar_df.shape}")
except FileNotFoundError:
    print("\n⚠️  Crop calendar not found")
    calendar_df = None

print("\n" + "="*60)
print("Sample weather data:")
print(meteo_df.head())

In [ ]:
# ============================================================
# LOAD FUTURES DATA
# ============================================================

def clean_futures(df):
    """Clean Investing.com downloaded data."""
    df = df.copy()
    
    # Standardize column names
    df.columns = [c.lower().strip().replace(' ', '_').replace('.', '').replace('%', 'pct') 
                  for c in df.columns]
    
    # Parse date with multiple formats
    df['date'] = pd.to_datetime(df['date'], format='mixed', dayfirst=False, errors='coerce')
    
    # If still NaT, try other formats
    if df['date'].isna().any():
        mask = df['date'].isna()
        df.loc[mask, 'date'] = pd.to_datetime(df.loc[mask, 'date'], format='%d/%m/%Y', errors='coerce')
    
    # Clean numeric columns
    for col in ['price', 'open', 'high', 'low']:
        if col in df.columns:
            df[col] = df[col].astype(str).str.replace(',', '').astype(float)
    
    # Rename price -> close
    df = df.rename(columns={'price': 'close'})
    
    # Sort by date
    df = df.sort_values('date').reset_index(drop=True)
    
    return df

print("Loading futures data...\n")
futures_df = pd.read_csv(FUTURES_PATH)
futures_clean = clean_futures(futures_df)

print(f"✓ Futures data: {futures_clean.shape}")
print(f"  Date range: {futures_clean['date'].min()} to {futures_clean['date'].max()}")
print(f"  Price range: €{futures_clean['close'].min():.2f} - €{futures_clean['close'].max():.2f}")
print(f"\nSample:")
print(futures_clean.head())

## 2. Data Processing Functions

In [ ]:
# ============================================================
# DATA PROCESSING FUNCTIONS
# ============================================================

def convert_date_format(df, date_col='date'):
    """Convert date column from int (YYYYMMDD) to datetime64."""
    df = df.copy()
    if df[date_col].dtype == 'int64':
        df[date_col] = pd.to_datetime(df[date_col].astype(str), format='%Y%m%d')
    elif df[date_col].dtype != 'datetime64[ns]':
        df[date_col] = pd.to_datetime(df[date_col])
    return df


def interpolate_to_daily(df, date_col='date', value_col='fpar', group_col='adm_id'):
    """
    Interpolate sparse time series (dekadal/8-day) to daily resolution.
    """
    print(f"  Interpolating {value_col} to daily...")
    
    df = df.copy()
    df = convert_date_format(df, date_col)
    
    # Get date range
    min_date = df[date_col].min()
    max_date = df[date_col].max()
    
    print(f"    Original: {len(df)} observations")
    print(f"    Date range: {min_date.date()} to {max_date.date()}")
    
    # Interpolate for each region
    regions = df[group_col].unique()
    interpolated = []
    
    for region in regions:
        region_data = df[df[group_col] == region].copy()
        region_data = region_data.set_index(date_col)
        
        # Create daily index
        daily_index = pd.date_range(
            region_data.index.min(), 
            region_data.index.max(), 
            freq='D'
        )
        
        # Reindex and interpolate
        region_daily = region_data[[value_col]].reindex(daily_index)
        region_daily[value_col] = region_daily[value_col].interpolate(
            method='linear',
            limit_direction='both'
        )
        
        region_daily[group_col] = region
        region_daily = region_daily.reset_index()
        region_daily = region_daily.rename(columns={'index': date_col})
        
        interpolated.append(region_daily)
    
    result = pd.concat(interpolated, ignore_index=True)
    print(f"    Interpolated: {len(result)} observations")
    
    return result


def create_daily_targets(futures_df, horizon=20):
    """
    Create target prices: for each trading day, get price at t+horizon.
    """
    print(f"\nCreating t+{horizon} targets...")
    
    df = futures_df[['date', 'close']].copy()
    df = df.sort_values('date').reset_index(drop=True)
    
    # Shift to get future price
    df[f'target_price_t{horizon}'] = df['close'].shift(-horizon)
    df = df.rename(columns={'close': 'price_current'})
    
    # Calculate future return
    df[f'target_return_t{horizon}'] = (
        (df[f'target_price_t{horizon}'] / df['price_current']) - 1
    ) * 100
    
    valid_targets = df[f'target_price_t{horizon}'].notna().sum()
    print(f"  Total trading days: {len(df)}")
    print(f"  Valid targets: {valid_targets}")
    print(f"  (Last {horizon} days have no target)")
    
    return df


def merge_daily_features(meteo_df, soil_moisture_df, fpar_daily):
    """
    Merge all daily features into single dataframe.
    """
    print("\nMerging daily features...")
    
    # Start with meteo as base
    result = meteo_df.copy()
    result = convert_date_format(result)
    print(f"  Base (meteo): {result.shape}")
    
    merge_keys = ['adm_id', 'date']
    
    # Add soil moisture if available
    if soil_moisture_df is not None:
        soil_moisture_df = convert_date_format(soil_moisture_df)
        sm_cols = merge_keys + [c for c in soil_moisture_df.columns 
                                if c not in merge_keys]
        result = result.merge(soil_moisture_df[sm_cols], on=merge_keys, how='left')
        print(f"  + soil_moisture: {result.shape}")
    
    # Add interpolated fPAR
    if fpar_daily is not None:
        fpar_cols = merge_keys + ['fpar']
        result = result.merge(fpar_daily[fpar_cols], on=merge_keys, how='left')
        print(f"  + fpar (interpolated): {result.shape}")
    
    return result


def add_targets_to_features(features_df, targets_df, horizon=20):
    """
    Add futures price targets to feature dataframe.
    """
    print(f"\nAdding t+{horizon} targets to features...")
    
    target_cols = ['date', f'target_price_t{horizon}', 'price_current', 
                   f'target_return_t{horizon}']
    
    features_df = convert_date_format(features_df)
    result = features_df.merge(targets_df[target_cols], on='date', how='inner')
    
    print(f"  Before merge: {len(features_df)} rows")
    print(f"  After merge: {len(result)} rows")
    
    # Drop rows without valid target
    result = result.dropna(subset=[f'target_price_t{horizon}'])
    print(f"  After dropping NaN targets: {len(result)} rows")
    
    return result


print("✓ Processing functions defined")

## 3. Build Feature Matrix

In [ ]:
# ============================================================
# BUILD DAILY FEATURE MATRIX
# ============================================================

print("\n" + "="*60)
print("BUILDING DAILY FEATURE MATRIX")
print("="*60)

# Step 1: Interpolate fPAR to daily
print("\nStep 1: Interpolating vegetation indices...")
fpar_daily = interpolate_to_daily(fpar_df, value_col='fpar', group_col='adm_id')

# Step 2: Create futures targets
print("\nStep 2: Creating futures targets...")
targets = create_daily_targets(futures_clean, horizon=HORIZON)

# Step 3: Merge features
print("\nStep 3: Merging features...")
features = merge_daily_features(meteo_df, soil_df, fpar_daily)

# Step 4: Add targets
dataset = add_targets_to_features(features, targets, horizon=HORIZON)

# Step 5: Add time features
print("\nStep 5: Adding time features...")
dataset['year'] = dataset['date'].dt.year
dataset['month'] = dataset['date'].dt.month
dataset['day_of_year'] = dataset['date'].dt.dayofyear
dataset['week_of_year'] = dataset['date'].dt.isocalendar().week.astype(int)

print("\n" + "="*60)
print("✓ FEATURE MATRIX COMPLETE")
print("="*60)
print(f"Shape: {dataset.shape}")
print(f"Regions: {dataset['adm_id'].nunique()}")
print(f"Unique dates: {dataset['date'].nunique()}")
print(f"Date range: {dataset['date'].min()} to {dataset['date'].max()}")
print(f"Years: {sorted(dataset['year'].unique())}")

## 4. Data Exploration

In [ ]:
# ============================================================
# EXPLORATION FUNCTIONS
# ============================================================

def explore_temporal_coverage(df, date_col='date', title='Temporal Coverage'):
    """Analyze temporal distribution."""
    print(f"\n{'='*60}")
    print(f"{title}")
    print(f"{'='*60}")
    
    dates = pd.to_datetime(df[date_col])
    
    print(f"Date range: {dates.min()} to {dates.max()}")
    print(f"Total span: {(dates.max() - dates.min()).days} days")
    print(f"Unique dates: {dates.nunique():,}")
    print(f"Total records: {len(df):,}")
    
    # Yearly distribution
    print("\nRecords by year:")
    yearly = dates.dt.year.value_counts().sort_index()
    for year, count in yearly.items():
        print(f"  {year}: {count:,}")
    
    return dates


def explore_spatial_coverage(df, region_col='adm_id'):
    """Analyze spatial distribution."""
    print(f"\n{'='*60}")
    print("Spatial Coverage")
    print(f"{'='*60}")
    
    regions = df[region_col].unique()
    print(f"Total regions: {len(regions)}")
    
    # Records per region
    region_counts = df[region_col].value_counts()
    print(f"\nRecords per region:")
    print(f"  Mean: {region_counts.mean():.0f}")
    print(f"  Median: {region_counts.median():.0f}")
    print(f"  Min: {region_counts.min()}")
    print(f"  Max: {region_counts.max()}")
    
    print(f"\nTop 5 regions by records:")
    for region, count in region_counts.head().items():
        print(f"  {region}: {count:,}")
    
    return region_counts


def explore_feature_distributions(df, sample_size=5000):
    """Analyze feature distributions and correlations."""
    print(f"\n{'='*60}")
    print("Feature Analysis")
    print(f"{'='*60}")
    
    # Auto-detect numeric columns
    feature_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    exclude = ['year', 'month', 'day_of_year', 'week_of_year']
    feature_cols = [c for c in feature_cols if c not in exclude]
    
    print(f"Analyzing {len(feature_cols)} features")
    
    # Sample for speed
    sample_df = df[feature_cols].sample(min(sample_size, len(df)), random_state=RANDOM_SEED)
    
    # Basic statistics
    print("\nFeature statistics:")
    stats = sample_df.describe()
    print(stats)
    
    # Missing values
    print("\nMissing values:")
    missing = df[feature_cols].isnull().sum()
    missing = missing[missing > 0].sort_values(ascending=False)
    if len(missing) > 0:
        print(missing)
    else:
        print("  ✓ No missing values!")
    
    # Correlation with target
    target_cols = [c for c in df.columns if 'target' in c.lower() and 'price' in c.lower()]
    if len(target_cols) > 0:
        print(f"\nCorrelation with {target_cols[0]}:")
        target_corr = sample_df.corrwith(df[target_cols[0]].sample(len(sample_df), random_state=RANDOM_SEED))
        target_corr = target_corr.sort_values(ascending=False)
        print(target_corr.head(10))
    
    return stats, sample_df


def plot_futures_price_analysis(futures_df, save_dir=FIGURES_DIR):
    """Detailed futures price analysis."""
    print(f"\n{'='*60}")
    print("Futures Price Analysis")
    print(f"{'='*60}")
    
    df = futures_df.copy()
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date')
    
    # Price statistics
    print(f"\nPrice statistics (EUR/tonne):")
    print(f"  Mean: {df['close'].mean():.2f}")
    print(f"  Median: {df['close'].median():.2f}")
    print(f"  Std: {df['close'].std():.2f}")
    print(f"  Min: {df['close'].min():.2f}")
    print(f"  Max: {df['close'].max():.2f}")
    
    # Daily returns
    df['return'] = df['close'].pct_change() * 100
    print(f"\nDaily returns (%):")
    print(f"  Mean: {df['return'].mean():.3f}")
    print(f"  Std: {df['return'].std():.3f}")
    print(f"  Min: {df['return'].min():.2f}")
    print(f"  Max: {df['return'].max():.2f}")
    
    # Create plot
    fig, axes = plt.subplots(3, 2, figsize=(15, 12))
    
    # 1. Price over time
    axes[0, 0].plot(df['date'], df['close'], linewidth=0.8)
    axes[0, 0].set_title('Wheat Futures Price Over Time', fontsize=12, fontweight='bold')
    axes[0, 0].set_ylabel('Price (EUR/tonne)')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Price distribution
    axes[0, 1].hist(df['close'], bins=50, edgecolor='black', alpha=0.7)
    axes[0, 1].set_title('Price Distribution', fontsize=12, fontweight='bold')
    axes[0, 1].set_xlabel('Price (EUR/tonne)')
    axes[0, 1].set_ylabel('Frequency')
    
    # 3. Daily returns
    axes[1, 0].plot(df['date'], df['return'], linewidth=0.5, alpha=0.7)
    axes[1, 0].axhline(y=0, color='r', linestyle='--', alpha=0.5)
    axes[1, 0].set_title('Daily Returns', fontsize=12, fontweight='bold')
    axes[1, 0].set_ylabel('Return (%)')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Return distribution
    axes[1, 1].hist(df['return'].dropna(), bins=50, edgecolor='black', alpha=0.7)
    axes[1, 1].set_title('Return Distribution', fontsize=12, fontweight='bold')
    axes[1, 1].set_xlabel('Daily Return (%)')
    axes[1, 1].set_ylabel('Frequency')
    
    # 5. Seasonal pattern
    df['month'] = df['date'].dt.month
    monthly_avg = df.groupby('month')['close'].mean()
    axes[2, 0].bar(monthly_avg.index, monthly_avg.values)
    axes[2, 0].set_title('Average Price by Month', fontsize=12, fontweight='bold')
    axes[2, 0].set_xlabel('Month')
    axes[2, 0].set_ylabel('Avg Price (EUR/tonne)')
    axes[2, 0].set_xticks(range(1, 13))
    
    # 6. Volatility
    df['volatility'] = df['return'].rolling(30).std()
    axes[2, 1].plot(df['date'], df['volatility'], linewidth=0.8)
    axes[2, 1].set_title('30-Day Rolling Volatility', fontsize=12, fontweight='bold')
    axes[2, 1].set_ylabel('Volatility (% std)')
    axes[2, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_dir / 'futures_analysis.png', dpi=150, bbox_inches='tight')
    print(f"\n✓ Saved plot to {save_dir / 'futures_analysis.png'}")
    plt.show()


def plot_feature_correlations(df, sample_size=5000, save_dir=FIGURES_DIR):
    """Plot correlation matrix."""
    feature_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    exclude = ['year', 'month', 'day_of_year', 'week_of_year']
    feature_cols = [c for c in feature_cols if c not in exclude]
    
    # Sample for speed
    sample_df = df[feature_cols].sample(min(sample_size, len(df)), random_state=RANDOM_SEED)
    
    # Compute correlation
    corr = sample_df.corr()
    
    # Plot
    fig, ax = plt.subplots(figsize=(12, 10))
    sns.heatmap(corr, cmap='coolwarm', center=0, 
                square=True, linewidths=0.5, cbar_kws={"shrink": 0.8},
                ax=ax, vmin=-1, vmax=1)
    ax.set_title(f'Feature Correlation Matrix (n={len(sample_df):,})', 
                 fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(save_dir / 'correlation_matrix.png', dpi=150, bbox_inches='tight')
    print(f"✓ Saved correlation matrix to {save_dir / 'correlation_matrix.png'}")
    plt.show()


def plot_seasonal_patterns(df, feature_cols=None, save_dir=FIGURES_DIR):
    """Plot seasonal patterns for key features."""
    df = df.copy()
    df['month'] = pd.to_datetime(df['date']).dt.month
    
    if feature_cols is None:
        feature_cols = ['tmin', 'tmax', 'prec', 'fpar']
    
    available_features = [f for f in feature_cols if f in df.columns]
    
    if len(available_features) == 0:
        print("⚠️  No features available for seasonal plotting")
        return
    
    fig, axes = plt.subplots(len(available_features), 1, 
                            figsize=(12, 3*len(available_features)))
    if len(available_features) == 1:
        axes = [axes]
    
    for i, feature in enumerate(available_features):
        monthly_pattern = df.groupby('month')[feature].agg(['mean', 'std'])
        
        axes[i].plot(monthly_pattern.index, monthly_pattern['mean'], 
                    marker='o', linewidth=2, markersize=6)
        axes[i].fill_between(monthly_pattern.index,
                            monthly_pattern['mean'] - monthly_pattern['std'],
                            monthly_pattern['mean'] + monthly_pattern['std'],
                            alpha=0.3)
        axes[i].set_title(f'Seasonal Pattern: {feature}', 
                         fontsize=12, fontweight='bold')
        axes[i].set_xlabel('Month')
        axes[i].set_ylabel(feature)
        axes[i].set_xticks(range(1, 13))
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_dir / 'seasonal_patterns.png', dpi=150, bbox_inches='tight')
    print(f"✓ Saved seasonal patterns to {save_dir / 'seasonal_patterns.png'}")
    plt.show()


print("✓ Exploration functions defined")

In [ ]:
# ============================================================
# RUN COMPREHENSIVE EXPLORATION
# ============================================================

print("\n" + "="*60)
print("COMPREHENSIVE DATA EXPLORATION")
print("="*60)

# Dataset overview
print(f"\nDataset shape: {dataset.shape}")
print(f"Memory usage: {dataset.memory_usage(deep=True).sum() / 1e6:.1f} MB")

# Temporal coverage
explore_temporal_coverage(dataset, date_col='date', title='Dataset Temporal Coverage')
explore_temporal_coverage(futures_clean, date_col='date', title='Futures Temporal Coverage')

# Spatial coverage
explore_spatial_coverage(dataset, region_col='adm_id')

# Feature distributions
stats, sample_df = explore_feature_distributions(dataset)

# Futures analysis
plot_futures_price_analysis(futures_clean)

# Correlation analysis
print("\n📊 Generating correlation matrix...")
plot_feature_correlations(dataset)

# Seasonal patterns
print("\n📊 Generating seasonal patterns...")
plot_seasonal_patterns(dataset)

print("\n" + "="*60)
print("✓ EXPLORATION COMPLETE!")
print("="*60)

## 5. Save Processed Data

In [ ]:
# ============================================================
# SAVE PROCESSED DATA
# ============================================================

print("\n" + "="*60)
print("SAVING PROCESSED DATA")
print("="*60)

# Save full dataset
output_path = OUTPUT_DIR / f"france_wheat_daily_t{HORIZON}.csv"
dataset.to_csv(output_path, index=False)
print(f"\n✓ Saved full dataset to: {output_path}")
print(f"  Size: {output_path.stat().st_size / 1e6:.1f} MB")
print(f"  Rows: {len(dataset):,}")
print(f"  Columns: {len(dataset.columns)}")

# Save sample for quick testing
sample_size = min(SAMPLE_SIZE, len(dataset))
sample = dataset.sample(sample_size, random_state=RANDOM_SEED)
sample_path = OUTPUT_DIR / f"france_wheat_daily_t{HORIZON}_sample.csv"
sample.to_csv(sample_path, index=False)
print(f"\n✓ Saved sample to: {sample_path}")
print(f"  Size: {sample_path.stat().st_size / 1e6:.1f} MB")
print(f"  Rows: {len(sample):,}")

print("\n" + "="*60)
print("✓ ALL DONE!")
print("="*60)
print("\nNext steps:")
print("1. Review the exploration plots in:", FIGURES_DIR)
print("2. Load the processed data for modeling:")
print(f"   df = pd.read_csv('{output_path}')")
print("3. Build prediction models (LSTM, Transformer, etc.)")
print("4. Evaluate on held-out test set")

## 6. Quick Data Summary

In [ ]:
# ============================================================
# QUICK DATA SUMMARY
# ============================================================

print("\n" + "="*60)
print("FINAL DATASET SUMMARY")
print("="*60)

# Basic info
print(f"\n📊 Shape: {dataset.shape}")
print(f"📅 Date range: {dataset['date'].min()} to {dataset['date'].max()}")
print(f"🗺️  Regions: {dataset['adm_id'].nunique()}")
print(f"📆 Years: {sorted(dataset['year'].unique())}")

# Features
non_feature_cols = ['adm_id', 'date', 'year', 'month', 'day_of_year', 
                    'week_of_year', f'target_price_t{HORIZON}', 
                    'price_current', f'target_return_t{HORIZON}']
feature_cols = [c for c in dataset.columns if c not in non_feature_cols]

print(f"\n🎯 Target: target_price_t{HORIZON}")
print(f"📈 Features ({len(feature_cols)}): {feature_cols}")

# Sample data
print("\n📋 Sample data:")
display_cols = ['date', 'adm_id', 'tmin', 'tmax', 'prec', 'fpar', 
                'price_current', f'target_price_t{HORIZON}']
display_cols = [c for c in display_cols if c in dataset.columns]
print(dataset[display_cols].head(10))

# Data quality
print("\n✅ Data quality:")
missing = dataset.isnull().sum()
missing = missing[missing > 0]
if len(missing) > 0:
    print("Missing values:")
    print(missing)
else:
    print("✓ No missing values!")

# Target statistics
print(f"\n📊 Target statistics:")
print(f"  Current price range: €{dataset['price_current'].min():.2f} - €{dataset['price_current'].max():.2f}")
print(f"  Target price range: €{dataset[f'target_price_t{HORIZON}'].min():.2f} - €{dataset[f'target_price_t{HORIZON}'].max():.2f}")
print(f"  Return range: {dataset[f'target_return_t{HORIZON}'].min():.2f}% - {dataset[f'target_return_t{HORIZON}'].max():.2f}%")